In [ ]:
# Define useful paths
SCRATCH_ROOT = "/storage/vbutoi/scratch/ESE"
CONFIG_ROOT = "/storage/vbutoi/projects/ESE/configs"
import sys
sys.path.append('/storage/vbutoi/projects')
sys.path.append('/storage/vbutoi/projects/ESE')

# IonPy imports
from ionpy.util import Config

%load_ext yamlmagic
%load_ext autoreload
%autoreload 2

In [ ]:
%%yaml default_cfg 

experiment:
    _class: "ese.experiment.CalibrationExperiment"
    exp_root: '?'
    inference_seed: 40
    sys_paths:
        - "/storage/vbutoi/projects"
        - "/storage/vbutoi/projects/ESE"
    data_paths:
        - "/storage"
        - "/storage/vbutoi/datasets"

inference_data:
    split: '?'

log:
    root: '?'
    save_preds: False 
    log_interval: 10 
    log_pixel_stats: False 
    gether_inference_stats: True
    compute_global_metrics: False 

dataloader:
    batch_size: '?' 
    num_workers: '?' 
    pin_memory: True 

visualize:
    _class: ese.analysis.visualize.ShowSegmentationPredictions
    col_wrap: 8

In [ ]:
%%yaml model_cfg 

# For standard datasets
#####################################
model:
    checkpoint: '?'

In [ ]:
%%yaml metrics_cfg

metrics:
    Image_ECE:
        _fn: ese.metrics.calibration.image_ece_loss
        num_prob_bins: 15
        from_logits: True
        lower_threshold: 0.01
        batch_reduction: none
    Dice:
        _fn: ionpy.metrics.dice_score
        from_logits: True
        batch_reduction: none
    Accuracy:
        _fn: ionpy.metrics.pixel_accuracy
        from_logits: True
        batch_reduction: none
    Entropy:
        _fn: ese.metrics.attributes.entropy_score
        from_logits: True
        batch_reduction: none

## Gather Inference Options.

In [ ]:
%%yaml experiment_cfg 
############################################################################################################
group: "OCTA_TotalEval_wAvgEntropy"

log:
    verbose: 
        # - True 
        - False 

############################################################################################################
## EXPERIMENTAL VARIABLES
base_model: 
    - "/storage/vbutoi/scratch/ESE/training/older_runs/2024/08_August_2024/08_07_24_OCTA_FULLRES_CrossEntropy/20240807_144627-M576-c3a81247e7e30c6bf47c72748764a898"
    - "/storage/vbutoi/scratch/ESE/training/02_26_25_OCTA_ReducedCalibratorSuite_CrossEntropy"
    - "/storage/vbutoi/scratch/ESE/training/older_runs/2024/08_August_2024/08_07_24_OCTA_FULLRES_SoftDice/20240807_144440-P7H9-459da42e97888bccb0aa79c297f388cd"
    - "/storage/vbutoi/scratch/ESE/training/02_26_25_OCTA_ReducedCalibratorSuite_SoftDicePT"

model:
    checkpoint: 'min-val-ece_loss'

inference_data:
    split: 'val'

# Change this for debugging
dataloader:
    batch_size: 4
    num_workers: 2 

In [ ]:
# Get the configs for the different runs.
base_cfg = Config(default_cfg).update([
    model_cfg, 
    metrics_cfg
])

In [ ]:
# Local imports
from ionpy.experiment.generate_configs import get_inference_configs

# Get the different experiment cfg yamls.
updated_base_cfg, inf_cfgs = get_inference_configs(
    exp_cfg=experiment_cfg, 
    base_cfg=base_cfg,
    config_root=CONFIG_ROOT,
    scratch_root=SCRATCH_ROOT,
    add_date=True,
)

In [ ]:
len(inf_cfgs)

## Running Jobs

In [ ]:
# # ####### FOR DEBUGGIN
# from ionpy.slite import run_job
# from ionpy.analysis.inference import run_inference 

# run_job(
#     config=inf_cfgs[5], 
#     job_func=run_inference,
#     run_name='debug',
#     show_examples=True,
#     gpu='0',
# )

In [ ]:
%%yaml submit_cfg

mode: "local"
group: "inference"
add_date: True 
track_wandb: True
jobs_per_gpu: 1
scratch_root: "/storage/vbutoi/scratch/ESE"

In [ ]:
# FOR SUBMISSION
from ionpy.slite import submit_jobs
from ionpy.analysis.inference import run_inference 

submit_jobs(
    cfg_list=inf_cfgs,
    submit_cfg=submit_cfg,
    exp_cfg=experiment_cfg,
    base_cfg=updated_base_cfg,
    job_func=run_inference
)